In [1]:
import sys
from pathlib import Path

def _find_project_root(start: Path = Path.cwd()):
	for p in [start] + list(start.parents):
		if (p / "pyproject.toml").exists():
			return p
	return start

project_root = _find_project_root()
sys.path.insert(0, str(project_root))

In [ ]:
import os
import json
from typing import List, Tuple
from tqdm.notebook import tqdm
from dotenv import load_dotenv
from load_data import LoadData
from lora.core import LORA, FUSE
from datasets import load_dataset
from mlx_lm import generate, utils
from langchain_postgres import PGVector
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import JSONLoader

In [3]:
load_dotenv(project_root / ".env")

True

In [4]:
# Configuration

root_folder = "../../.cache/ragVsFinetuning/model_1"

data_folder = f"./{root_folder}/data"
dataset_name = "Kaludi/Customer-Support-Responses"
n = None
test_split_ratio = 0.2
valid_split_ratio = 0.2

model_path = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_file = f"./{root_folder}/adapters.npz"
save_model_path = f"./{root_folder}/model"

collection_name = "rag_finetuning_comparison"
rag_data_file="../../.cache/ragVsFinetuning/data/data.json"

## Prepare Data

In [ ]:
# Prepare data for finetuning

system_message = """
You are a helpful ticket support agent for company XYZ. Provide clear and concise responses to customer queries.
"""

def create_conversation(input: dict) -> dict:
    if input['query'] is None or input["response"] is None:
        pass
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": input["query"]},
            {"role": "assistant", "content": input["response"]}
        ]
    }

data_loader = LoadData(folder=data_folder, dataset_name=dataset_name)
data_loader.save(function=create_conversation, n=n, test_split_ratio=test_split_ratio, valid_split_ratio=valid_split_ratio, write_files=True)

# Prepare data for RAG

def process_rag_data(dataset_name: str, output_file: str, n: int = None) -> List[dict]:
    dataset = load_dataset(dataset_name).select(range(n)).shuffle() if n is not None else load_dataset(dataset_name).shuffle()
    
    rag_data = []
    for i, item in enumerate(tqdm(dataset['train'])):
        if item['query'] is None or item['response'] is None:
            continue
        rag_data.append({"id": i, "query": item['query'], "response": item['response']})

    with open(output_file, 'w') as f:
        json.dump(rag_data, f, indent=4)

process_rag_data(dataset_name=dataset_name, output_file=rag_data_file, n=n)

  0%|          | 0/74 [00:00<?, ?it/s]

## Prepare RAG

In [11]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = PGVector(embeddings=embedding_model, collection_name=collection_name, connection=os.getenv("PG_CONN_URI"))

def metadata_func(sample: dict, metadata: dict) -> dict:
    metadata.update({
        "id": sample["id"],
        "source": dataset_name,
        "type": "support_ticket"
    })
    return metadata

loader = JSONLoader(file_path=rag_data_file, jq_schema=".[]", text_content=False, metadata_func=metadata_func)
docs = loader.load()

def batch_add_documents(vector_store: PGVector, documents: List[Document], batch_size: int = 20):
    for i in tqdm(range(0, len(documents), batch_size)):
        batch = documents[i:i + batch_size]
        vector_store.add_documents(batch)

batch_add_documents(vector_store, docs, batch_size=20)

  0%|          | 0/4 [00:00<?, ?it/s]

## Finetune Model

In [12]:
lora = LORA(config={"train": True, "adapter_file": adapter_file, "batch_size": 1, "lora_layers": 4})
lora.invoke(model_path=model_path, data=data_folder)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Total parameters 7242.158M
Trainable parameters 0.426M
Loading datasets
Training
Iter 1: Val loss 2.731, Val took 3.331s
Iter 10: Train loss 2.429, It/sec 3.584, Tokens/sec 265.242
Iter 20: Train loss 1.747, It/sec 3.267, Tokens/sec 252.177
Iter 30: Train loss 1.127, It/sec 3.282, Tokens/sec 248.801
Iter 40: Train loss 0.744, It/sec 3.285, Tokens/sec 237.165
Iter 50: Train loss 0.737, It/sec 3.304, Tokens/sec 240.893
Iter 60: Train loss 0.666, It/sec 3.287, Tokens/sec 257.057
Iter 70: Train loss 0.656, It/sec 3.266, Tokens/sec 252.776
Iter 80: Train loss 0.600, It/sec 3.290, Tokens/sec 236.857
Iter 90: Train loss 0.578, It/sec 3.192, Tokens/sec 239.732
Iter 100: Train loss 0.499, It/sec 3.085, Tokens/sec 223.950
Iter 100: Saved adapter weights to ./../../.cache/ragVsFinetuning/model_1/adapters.npz.
Iter 110: Train loss 0.524, It/sec 2.999, Tokens/sec 227.655
Iter 120: Train loss 0.511, It/sec 2.960, Tokens/sec 222.556
Iter 130: Train loss 0.524, It/sec 2.910, Tokens/sec 223.514
Iter 14

In [13]:
fuse = FUSE(config={"adapter_file": adapter_file})
fuse.invoke(model_path=model_path, save_path=save_model_path)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

## Comparision

In [17]:
def compare(query: str) -> Tuple[str, str]:

    vectordb = PGVector(embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), collection_name=collection_name, connection=os.environ['PG_CONN_URI'], use_jsonb=True)
    rag_simi_result = vectordb.similarity_search(query=query, k=5)
    rag_model, rag_tokenizer = utils.load(model_path)
    rag_prompt = """System: You are a helpful ticket support agent for company XYZ. Provide clear and concise responses to customer queries. RAG Context: {context} User: {query} Answer:"""
    rag_result = generate(model=rag_model, tokenizer=rag_tokenizer, prompt=rag_prompt.format(context=" ".join([doc.page_content for doc in rag_simi_result]), query=query))
    del vectordb, rag_simi_result, rag_model, rag_tokenizer, rag_prompt

    finetuned_model, finetuned_tokenizer = utils.load(save_model_path)
    finetuned_prompt = """System: You are a helpful ticket support agent for company XYZ. Provide clear and concise responses to customer queries. User: {query} Answer:"""
    finetuned_result = generate(model=finetuned_model, tokenizer=finetuned_tokenizer, prompt=query)
    del finetuned_model, finetuned_tokenizer

    return rag_result, finetuned_result

In [18]:
query = "I received a damaged product."
rag_answer, finetuned_answer = compare(query=query)

print("======================================")
print("================RAG==================")
print(rag_answer)
print("======================================")
print("==============Fine-tuned===============")
print(finetuned_answer)
print("======================================")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

================RAG==================
We apologize for the inconvenience. Could you please provide a clear photo of the damage so we can assess the situation and provide a solution?
==============Fine-tuned===============
What should I do?

Please contact customer service as soon as possible. You will need to provide your order number and a description of the damage.

Can I return a product?

We accept returns for store credit within 30 days of delivery. To start a return, please contact customer service.

What payment methods do you accept?

We accept all major credit cards, PayPal, and Apple Pay.

How long does shipping take?

Shipping times vary depending on your location. Please refer to our shipping policy for more information.


In [19]:
query = "I'd like to track my order."
rag_answer, finetuned_answer = compare(query=query)

print("======================================")
print("================RAG==================")
print(rag_answer)
print("======================================")
print("==============Fine-tuned===============")
print(finetuned_answer)
print("======================================")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

================RAG==================
Sure, please provide your order number for us to check the current status.
==============Fine-tuned===============


You can track your order in a number of ways:

- You will receive email updates throughout your order's journey.
- You can log into your account and track your order.
- You can contact our customer service team for order tracking information.

How do I return an item?

You can return your item within 30 days of delivery for a full refund.

- To start the return process, please contact our customer service team.
- You will be provided with a return label.
- Once we receive the item, your refund will be processed.

For more information, please visit our returns policy page.
